In [10]:
from pipeline.main import SourceArango
from pipeline.spacy_singleton import get_nlp_model
nlp = get_nlp_model()

In [57]:
def iv(t):
    return t.is_alpha

source = SourceArango(user='root', pw='test', collection='sent', skip=20, limit=50, logging=True)

mails = []
mails_ = []
subs = []
froms = []
for mail in source:
    maildoc = source.db.collection('mails').get(mail['mail_ids'][0].replace('mails/', ''))
    tmp = []
    tmp_ = []
    subs.append(maildoc['parts'][-1]['subject'] if 'subject' in maildoc['parts'][-1] else '')
    froms.append(maildoc['parts'][-1]['head']['from'] if 'from' in maildoc['parts'][-1]['head'] else '')
    for p in maildoc['parts']:
        tmp += [t.lemma_ for t in nlp(p['body']) if t.is_alpha]
        tmp_.append(p['body'])
    mails.append(' '.join(tmp))
    mails_.append('====------......--------........--------........---------====='.join(tmp_))

    

In [34]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(mails)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
tf = tf_vectorizer.fit_transform(mails)
tf_feature_names = tf_vectorizer.get_feature_names()

In [35]:
n_topics = 5

# Run NMF
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


In [49]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

n_top_words = 15
display_topics(nmf, tfidf_feature_names, n_top_words)
print("=============")
#display_topics(lda, tf_feature_names, n_top_words)

Topic 0:
program enron pron business analyst leader unit associates experience hr speak new lead forward hemisphere
Topic 1:
attach question list thanks weekly report summary end week october credit regard distribution contact error
Topic 2:
website report var publish west view available weekly trade difficulty work datum list day morning
Topic 3:
john mark david michael greg scott richard paul laura james tim steven bob robert george
Topic 4:
download msn explorer free zachary glare great grant good golden goal george gilbert group general


In [46]:
preds = lda.transform(tf_vectorizer.transform(mails))
preds.argmax(axis=1)

array([3, 1, 4, 0, 4, 0, 2, 3, 1, 2, 1, 4, 0, 2, 1, 1, 2, 2, 3, 3, 3, 1, 3,
       0, 2, 4, 4, 2, 2, 3, 1, 4, 3, 0, 0, 0, 3, 1, 0, 4, 4, 1, 3, 1, 1, 4,
       2, 3, 4, 4])

In [60]:
for ip, p in enumerate(preds.argmax(axis=1)):
    if p == 1:
        #print(mails_[ip])
        print(subs[ip])
        print('======== '+str(preds[i])+' =========')

PUD No.1 of Clark County
======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========
E-mail System Shutdown NotificationImportance:	High
======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========
Weekly gulatory and Government Affairs port
======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========
California Department of Water sources
======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========
Trading Policy
======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========
Weekly gulatory port
======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========
1200 port
======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========

======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========
TRV Notification:  (West VaR - 11/21/2001)
======== [ 0.00139383  0.00138997  0.00139227  0.00139533  0.9944286 ] =========
Sent on behalf of Mar

In [42]:
for i in range(5):
    print(mails_[i])
    print('======== '+str(lda.transform(tf_vectorizer.transform([mails[i]])))+' =========')

Wanted to take a minute to thank each of you for your assistance during our 
preparation for the Wholesale Services portion of yesterday's First Quarter 
Earnings Release conference call and today's First Quarter Update in New York.

Much of the data we accumulate facilitates - 

preparation of discussion points for the Earnings Conference call (replay is 
available at www.Enron.com, "Investors" icon); and
preparation of supporting narrative for our Analyst Slide presentation.

For your convenience, a copy of today's New York slide presentation (copy is 
also available at www.Enron.com, "Investors" icon) is included as an 
attachment below.

Thanks again.

David Leboe
Investor Relations

======== [[ 0.00641215  0.00636017  0.0064      0.97445436  0.00637332]] =========
Shari,

Attached please find a credit worksheet for a draft EEI Master Power Purchase 
& Sale Agreement between EPMI and Public Utility District No. 1 of Clark 
County.

Please let me know if you have any questions regar